**Importing Libraries**

In [1]:
!pip install opencv-python-headless numpy --user

**Splitting The DataSet**

In [8]:
import os
import shutil
import random

# Define the paths to your dataset folders
dataset_dir = "dataset"  # Update this to the correct path
training_dir = "training"
testing_dir = "testing"
validating_dir = "validating"

# Define the ratio for splitting (80% training, 10% testing, 10% validating)
train_ratio = 0.8
test_ratio = 0.1
valid_ratio = 0.1

# Create the destination folders if they don't exist
os.makedirs(training_dir, exist_ok=True)
os.makedirs(testing_dir, exist_ok=True)
os.makedirs(validating_dir, exist_ok=True)

# Iterate through the "car" and "notcar" folders
categories = ["car", "notcar"]
for category in categories:
    category_dir = os.path.join(dataset_dir, category)  # Construct the full path to the category folder
    
    # Create subdirectories in the destination folders for "car" and "notcar"
    os.makedirs(os.path.join(training_dir, category), exist_ok=True)
    os.makedirs(os.path.join(testing_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validating_dir, category), exist_ok=True)
    
    # Get a list of all image files in the category folder
    image_files = os.listdir(category_dir)
    
    # Shuffle the image files randomly
    random.shuffle(image_files)
    
    # Calculate the number of images for each split
    num_images = len(image_files)
    num_train = int(num_images * train_ratio)
    num_test = int(num_images * test_ratio)
    
    # Split the images into training, testing, and validating sets
    train_images = image_files[:num_train]
    test_images = image_files[num_train:num_train + num_test]
    valid_images = image_files[num_train + num_test:]
    
    # Copy the images to their respective folders
    for image in train_images:
        src = os.path.join(category_dir, image)
        dst = os.path.join(training_dir, category, image)
        shutil.copy(src, dst)
    
    for image in test_images:
        src = os.path.join(category_dir, image)
        dst = os.path.join(testing_dir, category, image)
        shutil.copy(src, dst)
    
    for image in valid_images:
        src = os.path.join(category_dir, image)
        dst = os.path.join(validating_dir, category, image)
        shutil.copy(src, dst)

print("Data split and copied successfully.")


Data split and copied successfully.


**Training the Data() Version 1**
*This doesnt save the model*

In [9]:
import numpy as np
import cv2
import os
import time

# Function to extract HOG features from an image
def extract_features(image_path, fixed_length):
    # Load the image and resize it to a smaller size
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 128))  # Resize to a smaller size
    
    # Extract HOG features with reduced dimensionality
    hog = cv2.HOGDescriptor((64, 64), (16, 16), (8, 8), (8, 8), 9)
    features = hog.compute(image)
    
    # Convert the tuple to a NumPy array and then flatten
    features = np.array(features)
    features = features.flatten()
    
    # Resize features to a fixed length
    if len(features) < fixed_length:
        features = np.pad(features, (0, fixed_length - len(features)))
    elif len(features) > fixed_length:
        features = features[:fixed_length]
    
    return features

# Perceptron class for binary classification
class Perceptron:
    def __init__(self, num_features):
        self.weights = np.random.rand(num_features)
        self.bias = np.random.rand()
    
    def predict(self, features):
        activation = np.dot(features, self.weights) + self.bias
        return 1 if activation > 0 else 0
    
    def train(self, features, label, learning_rate=0.01):
        if len(features) != len(self.weights):
            raise ValueError("Number of features must match the number of weights")
        
        prediction = self.predict(features)
        error = label - prediction
        self.weights += learning_rate * error * features
        self.bias += learning_rate * error

# Path to the training data folders (car and not_car)
car_folder = "training/car"
not_car_folder = "training/car"

# Initialize the perceptron
fixed_length = 1000  # Adjust this value based on your needs
num_features = fixed_length
perceptron = Perceptron(num_features)

# Training loop
epochs = 10  # Adjust the number of epochs as needed
for epoch in range(epochs):
    start_time = time.time()
    correct_predictions = 0
    total_images = 0
    
    # Training on car images
    for image_name in os.listdir(car_folder):
        image_path = os.path.join(car_folder, image_name)
        features = extract_features(image_path, fixed_length)
        perceptron.train(features, 1)  # Label 1 for car images
        prediction = perceptron.predict(features)
        if prediction == 1:
            correct_predictions += 1
        total_images += 1

        # Training on not car images (alternating between car and not car)
        no_car_image_name = os.listdir(not_car_folder)[total_images % len(os.listdir(not_car_folder))]
        no_car_image_path = os.path.join(not_car_folder, no_car_image_name)
        features = extract_features(no_car_image_path, fixed_length)
        perceptron.train(features, 0)  # Label 0 for not car images
        prediction = perceptron.predict(features)
        if prediction == 0:
            correct_predictions += 1
        total_images += 1
    
    efficiency = correct_predictions / total_images
    end_time = time.time()
    training_time = end_time - start_time
    
    # Print training progress for each epoch
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Training time: {training_time:.2f} seconds")
    print(f"Efficiency: {efficiency:.2f}\n")


Epoch 1/10
Training time: 14.19 seconds
Efficiency: 0.54

Epoch 2/10
Training time: 6.16 seconds
Efficiency: 0.62

Epoch 3/10
Training time: 5.98 seconds
Efficiency: 0.69

Epoch 4/10
Training time: 6.08 seconds
Efficiency: 0.72

Epoch 5/10
Training time: 6.22 seconds
Efficiency: 0.75

Epoch 6/10
Training time: 5.95 seconds
Efficiency: 0.79

Epoch 7/10
Training time: 6.05 seconds
Efficiency: 0.79

Epoch 8/10
Training time: 6.25 seconds
Efficiency: 0.81

Epoch 9/10
Training time: 6.34 seconds
Efficiency: 0.82

Epoch 10/10
Training time: 6.63 seconds
Efficiency: 0.84



**Training the Data() Version 2**
*This saves the model*

In [7]:
import numpy as np
import cv2
import os
import time
import pickle  # Import the pickle library

def extract_features(image_path, fixed_length):
    # Load the image and resize it to a smaller size
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 128))  # Resize to a smaller size
    
    # Extract HOG features with reduced dimensionality
    hog = cv2.HOGDescriptor((64, 64), (16, 16), (8, 8), (8, 8), 9)
    features = hog.compute(image)
    
    # Convert the tuple to a NumPy array and then flatten
    features = np.array(features)
    features = features.flatten()
    
    # Resize features to a fixed length
    if len(features) < fixed_length:
        features = np.pad(features, (0, fixed_length - len(features)))
    elif len(features) > fixed_length:
        features = features[:fixed_length]
    
    return features

class Perceptron:
    def __init__(self, num_features):
        self.weights = np.random.rand(num_features)
        self.bias = np.random.rand()
    
    def predict(self, features):
        activation = np.dot(features, self.weights) + self.bias
        return 1 if activation > 0 else 0
    
    def train(self, features, label, learning_rate=0.01):
        if len(features) != len(self.weights):
            raise ValueError("Number of features must match the number of weights")car_folder
        
        prediction = self.predict(features)
        error = label - prediction
        self.weights += learning_rate * error * features
        self.bias += learning_rate * error

# Path to the training data folders
car_folder = "training/car"
not_car_folder = "training/car"

# Initialize the perceptron
fixed_length = 1000  # Adjust this value based on your needs
num_features = fixed_length

# Check if a saved model exists, if not, create a new one
if os.path.exists("perceptron_model.pkl"):
    with open("perceptron_model.pkl", "rb") as model_file:
        perceptron = pickle.load(model_file)
else:
    perceptron = Perceptron(num_features)

# Training loop
epochs = 10  # Adjust the number of epochs as needed
for epoch in range(epochs):
    start_time = time.time()
    correct_predictions = 0
    total_images = 0
    
    for image_name in os.listdir(car_folder):
        image_path = os.path.join(car_folder, image_name)
        features = extract_features(image_path, fixed_length)
        perceptron.train(features, 1)  # Label 1 for car images
        prediction = perceptron.predict(features)
        if prediction == 1:
            correct_predictions += 1
        total_images += 1

        # Now, switch to a no car image
        no_car_image_name = os.listdir(not_car_folder)[total_images % len(os.listdir(not_car_folder))]
        no_car_image_path = os.path.join(not_car_folder, no_car_image_name)
        features = extract_features(no_car_image_path, fixed_length)
        perceptron.train(features, 0)  # Label 0 for not car images
        prediction = perceptron.predict(features)
        if prediction == 0:
            correct_predictions += 1
        total_images += 1
    
    efficiency = correct_predictions / total_images
    end_time = time.time()
    training_time = end_time - start_time
    
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Training time: {training_time:.2f} seconds")
    print(f"Efficiency: {efficiency:.2f}\n")

# Save the trained model
with open("perceptron_model.pkl", "wb") as model_file:
    pickle.dump(perceptron, model_file)


Epoch 1/10
Training time: 22.91 seconds
Efficiency: 0.99

Epoch 2/10
Training time: 5.66 seconds
Efficiency: 0.99

Epoch 3/10
Training time: 4.54 seconds
Efficiency: 1.00

Epoch 4/10
Training time: 4.45 seconds
Efficiency: 1.00

Epoch 5/10
Training time: 5.23 seconds
Efficiency: 1.00

Epoch 6/10
Training time: 5.86 seconds
Efficiency: 1.00

Epoch 7/10
Training time: 5.12 seconds
Efficiency: 1.00

Epoch 8/10
Training time: 4.99 seconds
Efficiency: 1.00

Epoch 9/10
Training time: 4.45 seconds
Efficiency: 1.00

Epoch 10/10
Training time: 4.25 seconds
Efficiency: 1.00



**Validating The Data**

In [11]:
import shutil# Path to the validating data folders (car and not_car)
validating_car_folder = "validating/car"
validating_not_car_folder = "validating/notcar"

correct_predictions = 0
total_images = 0

# Create the output directories if they don't exist
output_car_folder = "validating/outputcar"
output_not_car_folder = "validating/outputnotcar"
os.makedirs(output_car_folder, exist_ok=True)
os.makedirs(output_not_car_folder, exist_ok=True)

with open("validating/answer.txt", "w") as output:
    for folder, output_folder in [(validating_car_folder, output_car_folder), (validating_not_car_folder, output_not_car_folder)]:
        label = 1 if "car" in folder else 0
        for image_name in os.listdir(folder):
            image_path = os.path.join(folder, image_name)
            features = extract_features(image_path, fixed_length)  # Provide fixed_length here
            prediction = perceptron.predict(features)
            
            output.write(f"Image: {image_name}, Predicted Label: {prediction}\n")
            
            if prediction == label:
                correct_predictions += 1
                # Move the image to the corresponding output folder
                output_image_path = os.path.join(output_folder, image_name)
                shutil.copy(image_path, output_image_path)
            
            total_images += 1

    efficiency = correct_predictions / total_images
    output.write(f"Efficiency: {efficiency:.2f}\n")
    output.write(f"Weights: {perceptron.weights}\n")
    output.write(f"Bias: {perceptron.bias}\n")


**Testing The Data(via images)**

In [10]:
import shutil

test_car_folder = "testing/car"
test_not_car_folder = "testing/notcar"
output_file = "testing/answer.txt"
output_car_folder = "testing/outputcar"
output_not_car_folder = "testing/outputnotcar"

correct_predictions = 0
total_images = 0

# Create the output directories if they don't exist
os.makedirs(output_car_folder, exist_ok=True)
os.makedirs(output_not_car_folder, exist_ok=True)

with open(output_file, "w") as output:
    for folder, output_folder in [(test_car_folder, output_car_folder), (test_not_car_folder, output_not_car_folder)]:
        label = 1 if "car" in folder else 0
        for image_name in os.listdir(folder):
            image_path = os.path.join(folder, image_name)
            features = extract_features(image_path, fixed_length)  # Provide fixed_length here
            prediction = perceptron.predict(features)
            
            output.write(f"Image: {image_name}, Predicted Label: {prediction}\n")
            
            if prediction == label:
                correct_predictions += 1
                # Move the image to the corresponding output folder
                output_image_path = os.path.join(output_folder, image_name)
                shutil.copy(image_path, output_image_path)
            
            total_images += 1

    efficiency = correct_predictions / total_images
    output.write(f"Efficiency: {efficiency:.2f}\n")
    output.write(f"Weights: {perceptron.weights}\n")
    output.write(f"Bias: {perceptron.bias}\n")


**Final Function To give Input via Video**

In [6]:
import cv2
import shutil

# Input video file
input_video_file = "input2.mp4"
output_car_folder = "testvideo/outputcar"
output_not_car_folder = "testvideo/outputnotcar"
output_file = "testvideo/output.txt"  # Define the output file

# Create the output directories if they don't exist
os.makedirs(output_car_folder, exist_ok=True)
os.makedirs(output_not_car_folder, exist_ok=True)

# Initialize video capture
cap = cv2.VideoCapture(input_video_file)
frame_number = 0

# Create a temporary directory to save frames
temp_frame_dir = "test/frames"
os.makedirs(temp_frame_dir, exist_ok=True)

with open(output_file, "w") as output:
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        
        # Break the loop if the video has ended
        if not ret:
            break
        
        # Save the frame as an image
        frame_number += 1
        frame_filename = f"frame_{frame_number:04d}.png"
        frame_path = os.path.join(temp_frame_dir, frame_filename)
        cv2.imwrite(frame_path, frame)
        
        # Check if the frame was successfully saved
        if os.path.isfile(frame_path):
            # Extract features and predict
            features = extract_features(frame_path, fixed_length)  # Provide fixed_length here
            prediction = perceptron.predict(features)
            
            output.write(f"Frame: {frame_filename}, Predicted Label: {prediction}\n")
            
            # Move the frame to the corresponding output folder
            if prediction == 1:
                output_image_path = os.path.join(output_car_folder, frame_filename)
            else:
                output_image_path = os.path.join(output_not_car_folder, frame_filename)
            
            shutil.copy(frame_path, output_image_path)
        else:
            print(f"Failed to save frame: {frame_filename}")

# Release the video capture
cap.release()
cv2.destroyAllWindows()
